# About

If 2D analysis also need to be done, this notebook can be used.

Main purpose of this notebook is simply retrieving 2D slices from 3D .nii files to utilize for 2D image classification tasks.<br/>
-> Find desired slice interval for desired axis for particular subject id.<br/>
-> Open 3 more files named AD, CN and MCI to save slices under train, test and val sets.

Ultimately, we will have '.jpg', '.png' or even '.pdf' (you can choose desired format) under files in particular format for PyTorch dataset lib is also expected.

In [52]:
import os
from pathlib import Path
from nibabel.testing import data_path
import nibabel as nib

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [63]:
class utils:
    def __init__(self):
        return
    

    def read_image(self, img_path):
        return nib.load(os.path.join(data_path, img_path)).get_fdata()


    ''' Find image in the given path '''
    def take_mri(self, subject_id, path):
        for root, dirs, files in os.walk(path):
            for file in files:
                if subject_id in file and file.endswith(".nii"):
                    return os.path.join(root, file)
            

    # TODO: try to normalize slices if possible
    #       .nii file sizes are different.
            # 43.5mb : (256, 256, 166)
            # 23.6mb : (192, 192, 160)
            # 47.2mb : (256, 256, 180) and more different sizes...

    def get_slices(self, subject_id, path, dataset, group, axis, start, end, format):

        img_path = self.take_mri(subject_id, os.path.join(path, dataset))
        img = self.read_image(img_path)
        

        for i in range(end - start):
            if axis == 'axial':
                slice = img[start + i,:,:]
            elif axis == 'coronal':
                slice = img[:,start + i,:]
            else: # sagittal
                slice = img[:,:,start + i]

            save_path = f'{path}/{dataset}/{group}/'
            os.makedirs(save_path, 0o755, True)
            plt.imsave(f'{save_path}{subject_id}-{axis}Slice{start + i}{format}', slice, cmap='gray')



    def run(self, data, path, dataset, axis = 'axial', start = 95, end = 100, format = '.jpg'):
        assert dataset in ['train', 'test', 'val']
        assert format in ['.png', '.jpg', '.pdf']
        assert axis in ['axial', 'sagittal', 'coronal', 'all']
        #TODO: assert start and end points.
        
        # DO NOT FORGET THIS RANGE IS NORMALLY -> len(data['subject'])
        for i in range(35):
            if axis == 'axial':
                self.get_slices(subject_id = data['subject'][i], path = path, dataset = dataset, 
                    group = data['group'][i], axis = 'axial', start = start, end = end, format = format)
            elif axis == 'coronal':
                self.get_slices(subject_id = data['subject'][i], path = path, dataset = dataset, 
                    group = data['group'][i], axis = 'coronal', start = start, end = end, format = format)
            elif axis == 'sagittal':
                self.get_slices(subject_id = data['subject'][i], path = path, dataset = dataset, 
                    group = data['group'][i], axis = 'sagittal', start = start, end = end, format = format)
            else: # 'all'
                self.get_slices(subject_id = data['subject'][i], path = path, dataset = dataset, 
                    group = data['group'][i], axis = 'axial', start = start, end = end, format = format)
                self.get_slices(subject_id = data['subject'][i], path = path, dataset = dataset, 
                    group = data['group'][i], axis = 'coronal', start = start, end = end, format = format)
                self.get_slices(subject_id = data['subject'][i], path = path, dataset = dataset, 
                    group = data['group'][i], axis = 'sagittal', start = start, end = end, format = format)

In [64]:
u = utils()

In [55]:
path = '/Users/toygar/Desktop/Bitirme/data/'
train = pd.read_csv(os.path.join(path, 'train/train.csv'))

In [66]:
u.run(train, path, 'train', axis = 'axial', start = 95, end = 97, format = '.jpg')
u.run(train, path, 'train', axis = 'coronal', start = 95, end = 97, format = '.jpg')
u.run(train, path, 'train', axis = 'sagittal', start = 95, end = 97, format = '.jpg')